Подготовить python-скрипт извлечения данных из файла 01_02_flowvariables.xlsx. <br> Скрипт должен извлечь
данные со всех листов кроме листа «01_02_00». С каждого листа следует извлечь табличные данные без сносок
и пустых строк, также следует извлечь наименование листа и наименование показателя (ячейка A2). <br>
Горизонтальная структура таблиц должна быть трансформирована в вертикальную, таким образом, чтобы
заголовки колонок-дат образовали новую вертикальную атрибут-колонку. <br> Полученный результат:
1. Region – наименование региона
2. date – отчётный месяц показателя
3. value – значение показателя
4. val_name – наименование показателя
5. sheet_name – наименование листа

In [9]:
import pandas as pd
xl = pd.ExcelFile('01_02_flowvariables.xlsx')
sheets = xl.sheet_names
sheets.remove('01_02_00')
data = pd.DataFrame()
for sheet_name in sheets:
    print(sheet_name)
    list = xl.parse(sheet_name)
    val_name = list.iloc[0, 0] #из 1 строки 2 столбца достаём название, куда-то надо записать
    list = list.dropna()
    list.reset_index(drop=True, inplace=True)
    list.rename(columns=list.iloc[0], inplace = True)
    list.drop([0], inplace = True)
    #list.reset_index(drop=True, inplace=True)
    list.set_index('Регион', inplace=True)
    list = list.unstack().reset_index(name='value')
    list['val_name']=val_name
    list['sheet_name']=sheet_name
    data = pd.concat([data, list]) 
data = data.rename(columns={'level_0': 'date', 'Регион': 'Region', })
data = data.reindex(columns=['Region','date','value','val_name','sheet_name'])
data.head(5)

01_02_01
01_02_02
01_02_03
01_02_04
01_02_05
01_02_06
01_02_07
01_02_08
01_02_09
01_02_10
01_02_11
01_02_12
01_02_13
01_02_14
01_02_15
01_02_16
01_02_17
01_02_18
01_02_19
01_02_20
01_02_21
01_02_22
01_02_23
01_02_24
01_02_25
01_02_26
01_02_27
01_02_28
01_02_29
01_02_30
01_02_31
01_02_32
01_02_33
01_02_34
01_02_35
01_02_36
01_02_37
01_02_38
01_02_39
01_02_40


,Region,date,value,val_name,sheet_name
0,Российская Федерация,2020-01-01,102,Количество многоквартирных домов в составе про...,01_02_01
1,Центральный ФО,2020-01-01,24,Количество многоквартирных домов в составе про...,01_02_01
2,Белгородская область,2020-01-01,0,Количество многоквартирных домов в составе про...,01_02_01
3,Брянская область,2020-01-01,1,Количество многоквартирных домов в составе про...,01_02_01
4,Владимирская область,2020-01-01,2,Количество многоквартирных домов в составе про...,01_02_01


В базе данных есть таблица sandpit.test_table. <br> Она содержит сведения о строящихся на текущий момент
многоквартирных домах. <br> Поля этой таблицы: <br>
Наименование поля Тип Описание <br>
obj_id int8 Идентификатор жилого дома. Уникальный ключ таблицы <br>
obj_square float8 Площадь жилого дома, тыс. кв. м <br>
region_id int4 Идентификатор субъекта РФ, в котором строится жилой дом <br>
region_name text Наименование субъекта РФ, в котором строится жилой дом <br>
dev_id text Идентификатор девелопера, строящего жилой дом <br>
dev_name text Наименование девелопера, строящего жилой дом <br>
Подготовьте один sql-запрос, который выгружает таблицу с полями: <br>
- Рейтинг региона по объему жилищного строительства <br>
- Наименование региона <br>
- Объем жилищного строительства в регионе, в тыс. кв. м <br>
- Количество девелоперов, занимающихся строительством в регионе <br>
- В т.ч. количество девелоперов, строящих более 1 жилого дома в регионе <br>
- Наименование девелопера с наибольшим объемом жилищного строительства в регионе <br>
- В т.ч. доля его объем жилищного строительства в регионе, в % <br>

``` mysql
with
q as
    (
    Select 
        sub_q.*,
        max(amount) over (partition by region_id) as max_amount_p_reg -- максимальный объём строительства в регионе
    from	
        (
        Select
            region_id,
            region_name, -- считаю, что region_id к region_name 1:1
            dev_id,
            dev_name, -- считаю, что dev_id к dev_name 1:1
            count(obj_id) as objs_p_dev_in_reg, -- кол-во объектов у девелопера в регионе
            sum(obj_square) as amount -- объём строительства у девелопера в регионе
        from
            sandpit.test_table
        group by 
            1,2,3,4
        ) sub_q
    ),
reg_q as
    (
    Select
        region_id,
        region_name,
        count(dev_id) as developers_uniq_qnt, -- Количество девелоперов, занимающихся строительством в регионе
        -- distinct не нужен, поскольку ранее применили группировку, дающую уникальность
        count(case when objs_p_dev_in_reg>1 then dev_id) as developers_with_2_and_more_objs_uniq_qnt, -- количество девелоперов, строящих более 1 жилого дома в регионе
        sum(amount) as region_amount -- объем жилищного строительства в регионе
    from
        dev_q
    group by
        region_id,
        region_name
    ),
max_dev as
    (
    Select 
        region_id, 
        dev_name, -- это застройщик с максимальным объёмом в регионе ввиду условия в where
        amount as dev_amount_in_reg
    from
        q
    Where amount = max_amount_p_reg -- считаю, что только у одного девелопера можно получить максимальный объём в регионе
    -- можно исключить кейс с несколькими застройщиками с максимумом в регионе с помощью row_number вместо max в cte "q", таким образом выбирая либо случайного зайстройщика из тех, у кого максимум, или же в каком-то порядке (к примеру, алфавитном)
    )
    
Select
    rank() over (order by t1.region_amount desc) as rating, -- Рейтинг региона по объему жилищного строительства
    t1.region_name, -- Наименование региона
    t1.region_amount, -- Объем жилищного строительства в регионе, в тыс. кв. м
    t1.developers_uniq_qnt, -- Количество девелоперов, занимающихся строительством в регионе
    t1.developers_with_2_and_more_objs_uniq_qnt, -- В т.ч. количество девелоперов, строящих более 1 жилого дома в регионе
    t2.dev_name, -- Наименование девелопера с наибольшим объемом жилищного строительства в регионе
    CONCAT(cast(round(t2.dev_amount_in_reg/t1.region_amount*100, 0) as String), '%') as max_dev_share -- В т.ч. доля его объем жилищного строительства в регионе, в %
from 
    reg_q t1
join
    max_dev t2
on t1.region_id = t2.region_id
```